In [210]:
# import libraries
import pprint
import requests
import re
import pyjq
import json
from newspaper import Article
import pymongo
import datetime
from dateutil.rrule import rrule, MONTHLY
import math
import time

client = pymongo.MongoClient (host="da1.eecs.utk.edu")
db = client ['GDA']
coll = db ['NYT']
#clean this up for now
coll.drop()

start = 1960
end = 2020
years = []
for i in range(end-start):
    years.append(start + i)

api_key = 'mXe9rrak6WGZA7H2zzeGejAbh0LoCAzY'

for i in range(len(years)):    
    url_out = 'https://api.nytimes.com/svc/search/v2/articlesearch.json?q=student+loan&fq=pub_year:("'+str(years[i])+'")&api-key=' + api_key
    r = requests .get(url_out)
    json_data = r.json()
    meta = pyjq.all('.response .meta',json_data)
    pages = math.ceil(meta[0]['hits']/10)
    print(years[i])
    for p in range(pages):   
        url_in = 'https://api.nytimes.com/svc/search/v2/articlesearch.json?q=student+loan&fq=pub_year:("'+str(years[i])+'")&page='+str(p)+'&api-key='+ api_key
        print('Page {0} of {1}'.format(p,pages))
        r = requests .get(url_in)
        json_data = r.json()
        count = pyjq.all('.response .docs | length',json_data)[0]
        urls = pyjq.all('.response .docs[] .web_url',json_data)

        for j in range(count):
            toi_article = Article(urls[j], language="en")
            try:
                toi_article.download() 
            except:
                continue
            try:
                toi_article.parse() 
            except:
                continue
            try:
                toi_article.nlp() 
            except:
                continue
            coll.insert_one ( {'title':toi_article.title,
                   'text': toi_article.text,
                   'summary': toi_article.summary, 
                   'keywords': toi_article.keywords, 
                   'year':years[i],
                   'url': urls[j] } )
        time.sleep(6)

1960
Page 0 of 2
Page 1 of 2
1961
Page 0 of 1
1962
Page 0 of 1
1963
Page 0 of 1
1964
Page 0 of 22
Page 1 of 22
Page 2 of 22
Page 3 of 22
Page 4 of 22
Page 5 of 22
Page 6 of 22
Page 7 of 22
Page 8 of 22
Page 9 of 22
Page 10 of 22
Page 11 of 22
Page 12 of 22
Page 13 of 22
Page 14 of 22
Page 15 of 22
Page 16 of 22
Page 17 of 22
Page 18 of 22
Page 19 of 22
Page 20 of 22
Page 21 of 22


In [137]:
client = pymongo.MongoClient (host="da1.eecs.utk.edu")
db = client ['GDA']
coll = db ['NYT']

print('Total NYT articles {0}'.format(str(coll.count_documents({}))))

Total NYT articles 19738


In [ ]:
client = pymongo.MongoClient (host="da1.eecs.utk.edu")
db = client ['GDA']
coll = db ['NYT']
pp = pprint.PrettyPrinter(indent=1,width=65)
for r in coll. find():
  print(pp .pformat (r))

In [195]:
import pandas as pd

client = pymongo.MongoClient (host="da1.eecs.utk.edu")
db = client ['GDA']
coll = db ['NYT']

boom=[]
genX=[]
genY=[]
genZ=[]

cursor = coll.aggregate( [{ '$match': { 'year' : { '$gte' : 1960, '$lte' : 1964 } } },{'$count': 'Article Count'}] )
boom = list(cursor)
cursor = coll.aggregate( [{ '$match': { 'year' : { '$gte' : 1965, '$lte' : 1979 } } },{'$count': 'Article Count'}] )
genX = list(cursor)
cursor = coll.aggregate( [{ '$match': { 'year' : { '$gte' : 1980, '$lte' : 1999 } } },{'$count': 'Article Count'}] )
genY = list(cursor)
cursor = coll.aggregate( [{ '$match': { 'year' : { '$gte' : 2000, '$lte' : 2019 } } },{'$count': 'Article Count'}] )
genZ = list(cursor)

df = pd.DataFrame(boom+genX+genY+genZ, index =['Boomers: 1960 to 1964',
                                               'Gen X:   1965 to 1979',
                                               'Gen Y:   1980 to 2000',
                                               'Gen Z:   2000 to 2019'],
                  columns =['Article Count']) 
display(df)
display(df.sum())

,Article Count
Boomers: 1960 to 1964,270
Gen X: 1965 to 1979,2810
Gen Y: 1980 to 2000,6890
Gen Z: 2000 to 2019,9768


Article Count    19738
dtype: int64

In [211]:
def get_dataframe(year):
    import pandas as pd
    cursor = coll.aggregate( [{ '$match': { 'year' : { '$eq' : year} } }] )
    df = pd.DataFrame(list(cursor))
    return df

In [213]:
display(get_dataframe(1960))

,_id,keywords,summary,text,title,url,year
0,5dbafeb7baff234d2b05e63b,"[unavailable, 57, page, article, timesmachine,...","The New York Times ArchivesSeptember 27, 1960 ...",Credit... The New York Times Archives\n\nSepte...,U.S. SETS 57 MILLION FOR STUDENT LOANS,https://www.nytimes.com/1960/09/27/archives/us...,1960
1,5dbafeb8baff234d2b05e63c,"[23, plan, quit, page, vassar, 40, timesmachin...","The New York Times ArchivesMarch 23, 1960 , Pa...",Credit... The New York Times Archives\n\nMarch...,VASSAR SET TO QUIT STUDENT LOAN PLAN,https://www.nytimes.com/1960/03/23/archives/va...,1960
2,5dbafebbbaff234d2b05e63d,"[unavailable, timesmachineprinceton, pace, uni...",Full text is unavailable for this digitized ar...,Full text is unavailable for this digitized ar...,STUDENTS' LOANS REPORTED ON RISE; Aide at Prin...,https://www.nytimes.com/1960/07/31/archives/st...,1960
3,5dbafebcbaff234d2b05e63e,"[widen, plan, page, article, guaranteed, times...","The New York Times ArchivesMay 11, 1960 , Page...",Credit... The New York Times Archives\n\nMay 1...,STUDENT LOANS WIDEN; 20 Savings Banks Join Pla...,https://www.nytimes.com/1960/05/11/archives/st...,1960
4,5dbafebdbaff234d2b05e63f,"[oath, end, page, timesmachine, view, 1960, 19...","The New York Times ArchivesJanuary 19, 1960 , ...",Credit... The New York Times Archives\n\nJanua...,President Asks End Of Student-Loan Oath,https://www.nytimes.com/1960/01/19/archives/pr...,1960
5,5dbafebdbaff234d2b05e640,"[oath, unavailable, 21, loyalty, plan, page, a...","The New York Times ArchivesFebruary 21, 1960 ,...",Credit... The New York Times Archives\n\nFebru...,STUDENT LOAN PLAN; Wisconsin Will Accept Gifts...,https://www.nytimes.com/1960/02/21/archives/st...,1960
6,5dbafebebaff234d2b05e641,"[unavailable, program, page, timesmachine, vie...","The New York Times ArchivesFebruary 2, 1960 , ...",Credit... The New York Times Archives\n\nFebru...,EDUCATION AIDE NAMED; California Official to H...,https://www.nytimes.com/1960/02/02/archives/ed...,1960
7,5dbafebfbaff234d2b05e642,"[oath, meeting, unavailable, loyalty, plan, gr...","The New York Times ArchivesMarch 10, 1960 , Pa...",Credit... The New York Times Archives\n\nMarch...,COLLEGES ADVISED TO TRIM COURSES; Educator War...,https://www.nytimes.com/1960/03/10/archives/co...,1960
8,5dbafec0baff234d2b05e643,"[23, page, 360000, 62, article, timesmachine, ...","The New York Times ArchivesOctober 23, 1960 , ...",Credit... The New York Times Archives\n\nOctob...,"AMHERST STUDENT AID; $360,000 Is Given in Loan...",https://www.nytimes.com/1960/10/23/archives/am...,1960
9,5dbafec1baff234d2b05e644,"[unavailable, page, loans, article, timesmachi...","The New York Times ArchivesAugust 6, 1960 , Pa...",Credit... The New York Times Archives\n\nAugus...,MEDICAL STUDENT AID; Loans and Scholarships Ar...,https://www.nytimes.com/1960/08/06/archives/me...,1960
